## ДЗ по поиску

Привет! Вам надо реализивать поисковик на базе вопросов-ответов с сайта [pravoved.ru](https://pravoved.ru/questions-archive/).        
Поиск должен работать на трех технологиях:       
1. обратном индексе     
2. word2vec         
3. doc2vec      

Вы должны понять, какой метод и при каких условиях эксперимента на этом корпусе работает лучше.          
Для измерения качества поиска найдите точность (accuracy) выпадания правильного ответа на конкретный вопрос (в этой базе у каждого вопроса есть только один правильный ответ). Точность нужно измерить для всей базы.    
При этом давайте считать, что выпал правильный ответ, если он попал в **топ-5** поисковой выдачи.

> Сделайте ваш поиск максимально качественным, чтобы значение точности стремилось к 1.     
Для этого можно поэкспериментировать со следующим:       
- модель word2vec (можно брать любую из опен сорса или обучить свою)
- способ получения вектора документа через word2vec: простое среднее арифметическое или взвешивать каждый вектор в соответствии с его tf-idf      
- количество эпох у doc2vec (начинайте от 100)
- предобработка документов для обучения doc2vec (удалять / не удалять стоп-слова)
- блендинг методов поиска: соединить результаты обратного индекса и w2v, или (что проще) w2v и d2v

На это задание отведем 10 дней. Дэдлайн сдачи до полуночи 12.10.

In [1]:
import pickle

with open('qa_corpus.pkl', 'rb') as file:
    qa_corpus = pickle.load(file)

c:\py_37_env\lib\site-packages\ipykernel\parentpoller.py:116: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  ipython-dev@scipy.org""")


Всего в корпусе 1384 пары вопрос-ответ

In [4]:
type(qa_corpus)

list

Первый элемент блока это вопрос, второй - ответ на него

In [3]:
qa_corpus[1][0]

'\nЗдравствуйте, я продаю комнату в 4х комнатной квартире. С одним из соседей у меня долевка. Лицевые счета разные. Риелтор говорит что при продаже нужно будет оплачивать натариусу налог с дохода. 17400. Продала за 450000. Сама буду брать квартиру в ипотеку.\n'

## Word2vec

#### Начнем с модели из опенсорса без pos-tagging

In [2]:
import warnings
warnings.filterwarnings(action='ignore', module='gensim')

import gensim
from gensim.models import Word2Vec, KeyedVectors

In [3]:
from tqdm import tqdm_notebook as tqdm
import re
from judicial_splitter import splitter
import pandas as pd

In [4]:
model_path = 'araneum_none_fasttextcbow_300_5_2018/araneum_none_fasttextcbow_300_5_2018.model'
model = Word2Vec.load(model_path)

In [5]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string


def preprocessing(input_text, del_stopwords=True, del_digit=True):
    """
    :input: raw text
        1. lowercase, del punctuation, tokenize
        2. normal form
        3. del stopwords
        4. del digits
    :return: lemmas
    """
    russian_stopwords = set(stopwords.words('russian'))
    words = [x.lower().strip(string.punctuation+'»«–…') for x in word_tokenize(input_text)]
    lemmas = [morph.parse(x)[0].normal_form for x in words if x]

    lemmas_arr = []
    for lemma in lemmas:
        if del_stopwords:
            if lemma in russian_stopwords:
                continue
        if del_digit:
            if lemma.isdigit():
                continue
        lemmas_arr.append(lemma)
    return lemmas_arr

In [14]:
def no_spaces(text):
    
    processed_text = text.replace('\n', ' ').replace('\n\n', ' ').replace('ул. ', 'ул.').replace('г. ', 'г.').replace('гор. ', 'гор.').replace('с. ', 'с.')
    return processed_text


# убираем пробел после инициалов перед фамилией
def clear_abbrs(processed_text):
    initials = re.compile(r'[А-Я]{1}\.[А-Я]{1}\. [А-Я][а-яё]+')
    counter = len(initials.findall(processed_text))

    for s in range(counter):
        get_abbrs = initials.search(processed_text)
        i = get_abbrs.span()[0] + 4
        processed_text = processed_text[:i] + processed_text[i+1:]
    return processed_text

In [14]:
# this function works with preprocessing func result
def get_w2v_vectors(lemmas_arr, model):
    """Получает вектор документа"""
    if len(lemmas_arr) == 0:
        doc_vec = np.zeros(300)
    else:
        vectors = []
        for element in lemmas_arr:
            try:
                vec = model.wv[element]
                # len(vec) this gives us 300
            except KeyError:
                continue
            vectors.append(vec)
        vec_sum = np.zeros(300)
        for v in vectors:
            vec_sum += v
        doc_vec = vec_sum/len(vectors)   # усредненный вектор как опознаватель
    return doc_vec

In [15]:
# индексирует ответы
def save_w2v_base(qa_corpus, model):
    """Индексирует всю базу для поиска через word2vec"""
    col_names =  ['answer_vec', 'question', 'answer']
    dc  = pd.DataFrame(columns = col_names)
    for pair in tqdm(qa_corpus):
        question, answer = pair[0], pair[1]
        vecs = []
        for sentence in splitter(answer, 1):
            vec = get_w2v_vectors(preprocessing(sentence, del_stopwords=False), model)
            vecs.append(vec)
        vec = (np.array(vecs).sum(axis=0)/len(vecs)).tolist()
        dc.loc[len(dc)] = [vec, question, answer]
    return dc

In [16]:
dc = save_w2v_base(qa_corpus, model)

In [17]:
from gensim import matutils

def similarity(v1, v2):
    v1_norm = matutils.unitvec(np.array(v1))
    v2_norm = matutils.unitvec(np.array(v2))
    return np.dot(v1_norm, v2_norm)

In [15]:
def search_w2v(query, model_vectors, doc_text, model):
    """
    :doc_text: questions?
    :query: all answers
    """
    vecs = []
    for sentence in splitter(query, 1):
        vec = get_w2v_vectors(preprocessing(sentence, del_stopwords=False), model)
        vecs.append(vec)
    query_vector = (np.array(vecs).sum(axis=0)/len(vecs)).tolist()
    
    result = []
    i = 0
    for vec in model_vectors:
        similar = similarity(query_vector, vec)
        result.append((doc_text[i], similar))
        i += 1
    res = pd.DataFrame(result, columns=['answer','similarity'])
    return res.sort_values('similarity', ascending=False)  # sort by similarity

In [16]:
def accuracy_w2v(all_questions, model_vec, all_answers, model):
    correct = 0
    for i, q in tqdm(enumerate(all_questions)):
        res = list(search_w2v(q, model_vec, all_answers, model)[:5].answer)
        if all_answers[i] in res:
            correct += 1
    return correct / len(all_questions)

In [17]:
w2v_accuracy = accuracy_w2v(dc['question'], dc['answer_vec'], dc['answer'], model)
print(w2v_accuracy)


0.16979768786127167


In [6]:
# kernel постоянно умирает. Буду записывать результаты, чтобы в конце красиво вывести
w2v_accuracy = 0.16979768786127167

То же самое, но удаляя стоп-слова:

In [18]:
# индексирует ответы
def save_w2v_base2(qa_corpus, model):
    """Индексирует всю базу для поиска через word2vec"""
    col_names =  ['answer_vec', 'question', 'answer']
    dc  = pd.DataFrame(columns = col_names)
    for pair in tqdm(qa_corpus):
        question, answer = pair[0], pair[1]
        vecs = []
        for sentence in splitter(answer, 1):
            vec = get_w2v_vectors(preprocessing(sentence), model)
            vecs.append(vec)
        vec = (np.array(vecs).sum(axis=0)/len(vecs)).tolist()
        dc.loc[len(dc)] = [vec, question, answer]
    return dc

In [19]:
dc_sw = save_w2v_base2(qa_corpus, model)

In [21]:
def search_w2v(query, model_vectors, doc_text, model):
    """
    :doc_text: questions?
    :query: all answers
    """
    vecs = []
    for sentence in splitter(query, 1):
        vec = get_w2v_vectors(preprocessing(sentence), model)
        vecs.append(vec)
    query_vector = (np.array(vecs).sum(axis=0)/len(vecs)).tolist()
    
    result = []
    i = 0
    for vec in model_vectors:
        similar = similarity(query_vector, vec)
        result.append((doc_text[i], similar))
        i += 1
    res = pd.DataFrame(result, columns=['answer','similarity'])
    return res.sort_values('similarity', ascending=False)  # sort by similarity

In [22]:
w2v_accuracy_del_sw = accuracy_w2v(dc['question'], dc['answer_vec'], dc['answer'], model)
print(w2v_accuracy_del_sw)


0.1936416184971098


In [7]:
w2v_accuracy_del_sw = 0.1936416184971098

Можно разбить выборку и сравнивать вопросы с вопросами, а не ответы с вопросами. Например, 30% всего корпуса будут вопросы как бы от пользователя, а база, по которой ищем будет состоять из 70% корпуса. И тогда мы будем сравнивать вопросы пользователей с вопросами из базы. Но тогда мы не сможем посчитать accuracy, потому что правильный ответы будут как бы неизвестны.

Еще можно обучить свою модель word2vec, например, на части ответах из нашего корпуса.

In [23]:
documents = []
for el in tqdm(dc['answer']):
    q = preprocessing(el)
    documents.append(q)
    
my_model = Word2Vec(
    documents,
    size=300,
    window=10,
    min_count=2,
    workers=4)
my_model.train(documents, total_examples=len(documents), epochs=30)    

(3189765, 3672720)

In [24]:
dc2 = save_w2v_base(qa_corpus, my_model)

c:\py_37_env\lib\site-packages\ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide


In [23]:
dc2.head()

,answer_vec,question,answer
0,"[0.01078099909457652, 0.19173462382134268, 0.1...","\nДобрый день.Мой сын гражданин Украины (ДНР),...",Добрый вечер!Из Вашего вопроса вообще ничего н...
1,"[-0.4380003193808669, -0.03778596291620586, -0...","\nЗдравствуйте, я продаю комнату в 4х комнатно...","Оксана, Вы вправе не платить налог, если являе..."
2,"[0.2094762644586945, 0.08527215330722852, -0.1...",\nМожно ли подать приложения к жалобе по делу ...,"Здравствуйте, Илья! Можно ли подать приложения..."
3,"[0.04081736465818003, -0.07478159244515394, 0....",\nДобрый вечер.\r\nПроизошла мало приятная сит...,Представьте органу предварительного расследова...
4,"[0.831405877044388, -0.3886620435815037, 0.304...",\nЯ брала займ 5000 т.р произошли трудности в ...,"Прямо в полицию и обращайтесь, в заявлении про..."


In [25]:
my_accuracy = accuracy_w2v(dc2['question'], dc2['answer_vec'], dc2['answer'], my_model)
print(my_accuracy)

c:\py_37_env\lib\site-packages\ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide



0.18858381502890173


In [8]:
my_accuracy = 0.18858381502890173

## Doc2vec

In [29]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.test.utils import get_tmpfile

In [30]:
def train_doc2vec(data):
    tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(data)]
    model = Doc2Vec(vector_size=100, min_count=5, alpha=0.025, min_alpha=0.025, epochs=100, workers=4, dm=1)
    model.build_vocab(tagged_data)
    model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)
    return model

In [31]:
file_name = get_tmpfile("d2v_jura")
model_d2v = train_doc2vec(dc['answer'])
model_d2v.save(file_name)

In [32]:
model_d2v = Doc2Vec.load(file_name)

In [33]:
def get_d2v_vectors(lemmas_arr, model_d2v):
    """Получает вектор документа"""
    # model.infer_vector(["закон", "договор"])
    model_d2v.random.seed(100)  # ensure same results
    if len(lemmas_arr) == None:
        doc_vec = None
    else:
        doc_vec = model_d2v.infer_vector(lemmas_arr)
    return doc_vec

In [34]:
def save_d2v_base(qa_corpus, model_d2v):
    """Индексирует всю базу для поиска через doc2vec"""
    # индексируем ответы
    col_names =  ['answer_vec', 'question', 'answer']
    d  = pd.DataFrame(columns = col_names)
    for pair in tqdm(qa_corpus):
        question, answer = pair[0], pair[1]
        vecs = []
        for sentence in splitter(answer, 1):
            vec = get_d2v_vectors(preprocessing(sentence, del_stopwords=False), model_d2v)
            vecs.append(vec)
        vec = (np.array(vecs).sum(axis=0)/len(vecs)).tolist()
        d.loc[len(d)] = [vec, question, answer]
    return d

In [35]:
dc3 = save_d2v_base(qa_corpus, model_d2v)

In [36]:
dc3.head()

,answer_vec,question,answer
0,"[-2.062420606613159, -0.8957734107971191, -0.4...","\nДобрый день.Мой сын гражданин Украины (ДНР),...",Добрый вечер!Из Вашего вопроса вообще ничего н...
1,"[1.2384271621704102, -0.2944904565811157, 1.48...","\nЗдравствуйте, я продаю комнату в 4х комнатно...","Оксана, Вы вправе не платить налог, если являе..."
2,"[0.4222171902656555, -1.3481476306915283, 0.41...",\nМожно ли подать приложения к жалобе по делу ...,"Здравствуйте, Илья! Можно ли подать приложения..."
3,"[-0.7394524812698364, -4.3170013427734375, 1.2...",\nДобрый вечер.\r\nПроизошла мало приятная сит...,Представьте органу предварительного расследова...
4,"[-1.7640345096588135, -0.9663607478141785, -0....",\nЯ брала займ 5000 т.р произошли трудности в ...,"Прямо в полицию и обращайтесь, в заявлении про..."


In [37]:
def search_d2v(query, model_vectors, doc_text, model_d2v):
    # model_vectors = vectors from the base of answers saved into dc3
    vecs = []
    for sentence in splitter(query, 1):
        vec = get_d2v_vectors(preprocessing(sentence, del_stopwords=False), model_d2v)
        vecs.append(vec)
    query_vector = (np.array(vecs).sum(axis=0)/len(vecs)).tolist()
    
    result = []
    i = 0
    for vec in model_vectors:
        similar = similarity(query_vector, vec)
        result.append((doc_text[i], similar))
        i += 1
    res = pd.DataFrame(result, columns=['answer','similarity'])
    return res.sort_values('similarity', ascending=False)  # sort by similarity

In [38]:
l = dc3['answer'][1]

In [40]:
def accuracy_d2v(all_questions, model_vec, all_answers, model_d2v):
    correct = 0
    for i, q in tqdm(enumerate(all_questions)):
        res = list(search_d2v(q, model_vec, all_answers, model_d2v)[:5].answer)
        if all_answers[i] in res:
            correct += 1
    return correct / len(all_questions)

In [41]:
correct = accuracy_d2v(dc3['question'], dc3['answer_vec'], dc3['answer'], model_d2v)
print(correct)


0.19436416184971098


In [42]:
d2v_accuracy = correct
d2v_accuracy = 0.19436416184971098

Тот же самый doc2vec, но без стоп слов:

In [44]:
def save_d2v_base2(qa_corpus, model_d2v):
    """Индексирует всю базу для поиска через doc2vec"""
    # индексируем ответы
    col_names =  ['answer_vec', 'question', 'answer']
    d  = pd.DataFrame(columns = col_names)
    for pair in tqdm(qa_corpus):
        question, answer = pair[0], pair[1]
        vecs = []
        for sentence in splitter(answer, 1):
            vec = get_d2v_vectors(preprocessing(sentence), model_d2v)
            vecs.append(vec)
        vec = (np.array(vecs).sum(axis=0)/len(vecs)).tolist()
        d.loc[len(d)] = [vec, question, answer]
    return d

In [45]:
dc3 = save_d2v_base2(qa_corpus, model_d2v)

In [48]:
def search_d2v(query, model_vectors, doc_text, model_d2v):
    # model_vectors = vectors from the base of answers saved into dc3
    vecs = []
    for sentence in splitter(query, 1):
        vec = get_d2v_vectors(preprocessing(sentence), model_d2v)
        vecs.append(vec)
    query_vector = (np.array(vecs).sum(axis=0)/len(vecs)).tolist()
    
    result = []
    i = 0
    for vec in model_vectors:
        similar = similarity(query_vector, vec)
        result.append((doc_text[i], similar))
        i += 1
    res = pd.DataFrame(result, columns=['answer','similarity'])
    return res.sort_values('similarity', ascending=False)  # sort by similarity

In [49]:
correct = accuracy_d2v(dc3['question'], dc3['answer_vec'], dc3['answer'], model_d2v)
print(correct)


0.20881502890173412


In [50]:
d2v_accuracy_no_sw = correct
d2v_accuracy_no_sw = 0.20881502890173412

## Inverted Index

In [11]:
from collections import defaultdict
def inverted_index(texts) -> dict:
    """
    Create inverted index by input doc collection
    :return: inverted index
    """
    index = defaultdict(list)
    i = 0
    for text in texts:
        for word in text.split(' '):
            if i not in index[word]:
                index[word].append(i)
        i += 1
    return index

In [12]:
from math import log

def score_BM25(qf, dl, N, n, avgdl, k1=2.0, b=0.75) -> float:
    """
    Compute similarity score between search query and documents from collection
    :return: score
    """
    score = log((N - n + 0.5)/(n + 0.5)) * (k1 + 1) * qf / (qf + k1 * (1 - b + b * (dl / avgdl)))
    return score

In [18]:
av = []
for element in dc['answer']:
    av.append(len(element))
avgdl = round(sum(av) / len(av))
N = len(dc['answer'])

In [19]:
inv_ind = inverted_index(dc['answer'])

In [20]:
inv_ind['проблема']

[24, 477, 615, 1018, 1135]

In [21]:
def compute_sim(query, inv_ind, document, avgdl, N) -> float:
    """
    Compute similarity score between search query and documents from collection
    :return: score
    """
    if query in inv_ind.keys():
        n = len(inv_ind[query])
    else:
        n = 0
    qf = document.count(query)
    dl = len(document)
    scores = score_BM25(qf, dl, N, n, avgdl)
    return scores


def get_search_result(query, inv_ind, doc_text) -> list:
    """
    Compute sim score between search query and all documents in collection
    Collect as pair (doc_id, score)
    :param query: input text
    :return: list of lists with (doc_id, score)
    """
    lemmas_query = preprocessing(query, del_stopwords=False)
    result = []
    i = 0
    for document in doc_text:
        similar = 0
        for el in lemmas_query:
            similar += compute_sim(el, inv_ind, document, avgdl, N)
        result.append((doc_text[i], similar))
        i += 1
        
    res = pd.DataFrame(result, columns=['doc_text','similarity'])
    return res.sort_values('similarity', ascending=False)[:5]

In [22]:
dc['answer'][2]

'Здравствуйте, Илья!\xa0Можно ли подать приложения к жалобе по делу об административном правонарушении (штраф за парковку) (фотографии, копии-сканы документов и т.д.) для суда первой инстанции в электронном виде для уменьшения количества бумаги, например разместив их все на флешке или на DVD-диске?Да, это возможно в соответствии со ст. 35 ГПК РФ.ГПК РФ Статья 35. Права и обязанности лиц, участвующих в деле \xa0 ... 1.1. Лица, участвующие в деле, вправе представлять в суд документы как на бумажном носителе, так и в электронном виде, в том числе в форме электронного документа, подписанного электронной подписью в порядке, установленном\xa0законодательством\xa0Российской Федерации, заполнять форму, размещенную на официальном сайте суда в информационно-телекоммуникационной сети «Интернет». 1.2. Лица, участвующие в деле, вправе представлять в суд иные документы в электронном виде, в том числе в форме электронных документов, выполненных указанными лицами либо иными лицами, органами, организац

In [23]:
if dc['answer'][2] in list(get_search_result(dc['question'][2], inv_ind, dc['answer']).doc_text):
    print(True)

True


In [24]:
correct = 0
for i, q in tqdm(enumerate(dc['question'][:3])):
    print(i)
    res = list(get_search_result(q, inv_ind, dc['answer']).doc_text)
    if dc['answer'][i] in res:
        print(True)
        correct += 1

0
1
2
True



In [26]:
def accuracy_inv_ind(all_questions, inv_ind, all_answers):
    correct = 0
    for i, q in tqdm(enumerate(all_questions)):
        res = list(get_search_result(q, inv_ind, all_answers).doc_text)
        if all_answers[i] in res:
            correct += 1
    return correct / len(all_questions)

In [27]:
accuracy = accuracy_inv_ind(dc['question'], inv_ind, dc['answer'])

In [51]:
print('Inverted index method accuracy: ', accuracy)
print('Doc2vec method accuracy: ', d2v_accuracy)
print('Doc2vec method accuracy without stop-words: ', d2v_accuracy_no_sw)
print('Word2vec method accuracy: ', w2v_accuracy)
print('Word2vec method accuracy without stop-words: ', w2v_accuracy_del_sw)
print('Word2vec model trained on answers from corpus accuracy: ', my_accuracy)

Inverted index method accuracy:  0.3171965317919075
Doc2vec method accuracy:  0.19436416184971098
Doc2vec method accuracy without stop-words:  0.20881502890173412
Word2vec method accuracy:  0.16979768786127167
Word2vec method accuracy without stop-words:  0.1936416184971098
Word2vec model trained on answers from corpus accuracy:  0.18858381502890173


Обратный индекс имеет самую высокую точность. Doc2vec без стоп-слов на втором месте.

## Блендинг методов поиска